In [ ]:
#importo le librerie necessarie 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import pickle


#importo i csv e li inseriesco all'interno di due variabili
red_wine = pd.read_csv("https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/wine_quality/winequality-red.csv", sep=';')
white_wine = pd.read_csv("https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/wine_quality/winequality-white.csv", sep=';')

#aggiungo una colonna per entrambi i csv per riconoscere tra vini bianchi e rossi 
red_wine.insert(0, 'type', 0)
white_wine.insert(0, 'type', 1)

#unisco i csv in un unico dataframe
df_wine = pd.concat([red_wine, white_wine], ignore_index=True)

#stampo la variabile wine per assicurarmi che l'importo sia andato a buon fine
display(df_wine)

1 Fixed acidity. The predominant fixed acids in wine, such as tartaric, succinic, citric, and malic acids. 2 Volatile acidity. The high acetic acid present in wine, which causes an unpleasant vinegar taste. 3 Citric acid. A weak organic acid used to increase the freshness and flavor of wine. 4 Residual sugar. The amount of sugar left after fermentation. 5 Chlorides. The amount of salt in wine. The lower chloride rate creates better quality wines. 6 Free sulfur dioxide. SO2 is used for preventing wine from oxidation and microbial spoilage. 7 Total sulfur dioxide. The amount of free and bound forms of SO2. 8 Density. Depends on the alcohol and sugar content. Better wines usually have lower densities. 9 pH. Used to check the level of acidity or alkalinity of wine. 10 Sulfates. An antibacterial and antioxidant agent added to wine. 11 Alcohol. The percentage of alcohol in wine. A higher concentration leads to better quality. 12 Type. 0 Red wine, 1 White wine.

In [ ]:
#controllo i valori nulli
df_wine.isnull().sum()

In [ ]:
#visualizzo in tabelle ogni colonna del dataframe
df_wine.hist(figsize=(20,13))
plt.show()

In [ ]:
#heatmap correlazioni
plt.figure(figsize=(13, 10))
corr_matrix = df_wine.corr(numeric_only = True)

sns.heatmap(corr_matrix, annot=True)
plt.show()

Dal grafico si può notare una correlazione tra free_sulfur_dioxite e total_sulfur_dioxite, decido di eliminare total_sulfur_dioxite

In [ ]:
#elimino dalla tabella la colonna total_sulfur_dioxide
df_wine=df_wine.drop("total_sulfur_dioxide", axis=1)

In [ ]:
#grafico che conta i vini in base alla qualità

df_temp=df_wine['quality']
sns.displot(df_temp)
plt.show()

temp=df_temp.to_numpy()

plt.scatter(np.arange(temp.size), temp, alpha=0.2)
plt.show()

In [ ]:
corr_labels = corr_matrix['quality'].sort_values(ascending=False)
print(corr_labels)

In [ ]:
#ho creato una nuova colonna top quality assegna il valore 1 se la quality del vino è > 5 o il valore 0 se inferiore di 5
df_wine['top_quality'] = [1 if x > 5.8 else 0 for x in df_wine.quality]
display(df_wine)

In [ ]:
#selezione delle features e della colonna target
features = df_wine.drop(['quality', 'top_quality'], axis=1)
target = df_wine['top_quality']

#split di training e testing delle features e del target
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=40)

#shape delle tuple di training e di testing
x_train.shape, x_test.shape

In [ ]:
#normalizzazione dei valori
norm = MinMaxScaler()
xtrain = norm.fit_transform(x_train)
xtest = norm.transform(x_test)
print(xtrain)
print(xtest)

In [ ]:
#inizializzazione del modello di regressione logistica
logistic_model = LogisticRegression()

#training del modello
logistic_model.fit(xtrain, y_train)

#previsionI sui test
y_pred_logistic = logistic_model.predict(xtest)

#definizione della precisione del modello
accuracy = accuracy_score(y_test, y_pred_logistic)
print('Precisione del modello:', accuracy)

print('\n')

#inizializzazione della matrice di confusione
conf_matrix_logistic = confusion_matrix(y_test, y_pred_logistic)

#definizione dell'errore quadrato medio
mse = mean_squared_error(y_test, y_pred_logistic)
print('MSE:', mse)

#definizione di r2
r2 = r2_score(y_test, y_pred_logistic)
print('R2:', r2)

In [ ]:
pickle.dump(logistic_model,open("model.pk1","wb"))
model = pickle.load(open("model.pkl", "rb"))